In [50]:
import requests
import datetime

In [51]:
global_word_list : set[str] = set();

with open("words", "r") as word_file:
  for word in word_file:
    global_word_list.add(word.strip("\n"));

len(global_word_list)

14855

In [76]:
def filter_words(grey_word : list[str], green_word : list[str], yellow_word : list[str], word_list : set[str]):
  words_removed : int = 0
  words_to_remove : str[str] = set()

  for list_word in word_list:
    removed : bool = False

    for letter in grey_word:
      if letter in list_word:
        words_to_remove.add(list_word)
        removed = True
        break

    if removed:
      continue

    for i, letter in enumerate(green_word):
      if letter == ' ':
        continue

      if list_word[i] != letter:
        words_to_remove.add(list_word)
        removed = True
        break

    if removed:
      continue

    for i, letter in enumerate(yellow_word):
      if letter == ' ':
        continue

      if list_word[i] == letter:
        words_to_remove.add(list_word)
        removed = True
        break

      if not letter in list_word:
        words_to_remove.add(list_word)
        removed = True
        break


  for word in words_to_remove:
    word_list.remove(word)

  print(f"Removed {len(words_to_remove)} words")
  return word_list

In [53]:
def sort_words_by_frequencies(word_list : set[str]):
  frequencies : dict[int, int] = dict()
  for character in range(97, 123):
    frequencies[chr(character)] = 0

  for word in word_list:
    for letter in word:
      frequencies[letter] = frequencies[letter] + 1

  word_scores : dict[str, int] = dict()
  for word in word_list:
    word_scores[word] = 0
    letters_used_already : set[str] = set()
    for letter in word:
      if letter in letters_used_already:
        continue
      letters_used_already.add(letter)
      word_scores[word] = word_scores[word] + frequencies[letter]

  return word_scores

In [54]:
def get_next_best_word(frequency_dict : dict[str, int]):
  return max(frequency_dict, key=frequency_dict.get)

In [69]:
def get_wordle_answer(date : datetime.datetime):
  if date is None:
    date = datetime.datetime.now().date();
    
  response = requests.get(f"https://www.nytimes.com/svc/wordle/v2/{date.year}-{date.month:02}-{date.day:02}.json")
  if response.status_code != 200:
    raise IOError("Failed to get today's wordle answer")
    
  response_text = response.json()
  return response_text.get("solution")

In [56]:
def submit_guess(answer : str, guess: str):
  answer_copy = list(answer)
  guess_copy = list(guess)
  result_string : list[str] = ['0', '0', '0', '0', '0']

  for i in range(len(guess_copy)):
    letter = guess_copy[i]
    if answer_copy[i] == letter:
      result_string[i] = '2'
      guess_copy[i] = ' '
      answer_copy[i] = ' '

  for i in range(len(guess_copy)):
    letter = guess_copy[i]
    if letter == ' ':
      continue

    for j, answer_letter in enumerate(answer_copy):
      if letter == answer_letter:
        answer_copy[j] = ' '
        guess_copy[i] = ' '
        result_string[i] = '1'
        break

  return result_string
      

In [87]:
word_list_copy : set[str] = set()
for word in global_word_list:
  word_list_copy.add(word)

answer = get_wordle_answer(None)
print(f"Answer is {answer}")

guesses = 0;
while len(word_list_copy) != 1:
  guesses += 1
  if len(word_list_copy) == 0:
      print("No words found")
      break

  word_frequencies : dict[str, int] = sort_words_by_frequencies(word_list_copy)

  print(f"Possible words remaining: {len(word_list_copy)}")
  if len(word_list_copy) <= 10:
    for i, word in enumerate(reversed(sorted(word_frequencies, key=word_frequencies.get))):
      print(f"{i + 1} - {word} ({word_frequencies.get(word)})")

  guess = get_next_best_word(word_frequencies)
  print(f"Best guess is {guess}")

  wordle_string = submit_guess(answer, guess)
  print(f"Result is {wordle_string}")

  grey_letters : list[str] = [' ', ' ', ' ', ' ', ' ']
  yellow_letters : list[str] = [' ', ' ', ' ', ' ', ' ']
  green_letters : list[str] = [' ', ' ', ' ', ' ', ' ']

  for character_index in range(0, 5):
    if (wordle_string[character_index] == '0'):
      grey_letters[character_index] = guess[character_index]
    elif (wordle_string[character_index] == '1'):
      yellow_letters[character_index] = guess[character_index]
    elif (wordle_string[character_index] == '2'):
      green_letters[character_index] = guess[character_index]
    else:
      raise RuntimeError("String input incorrectly")
  
  word_list_copy = filter_words(grey_letters, green_letters, yellow_letters, word_list_copy)

print(f"The word is {list(word_list_copy)[0]}")
print(f"Guessed in {guesses}")


Answer is elude
Possible words remaining: 14855
Best guess is arose
Result is ['0', '0', '0', '0', '2']
Removed 14509 words
Possible words remaining: 346
Best guess is tuile
Result is ['0', '1', '0', '1', '2']
Removed 337 words
Possible words remaining: 9
1 - flume (34)
2 - glume (34)
3 - blume (34)
4 - plume (34)
5 - kluge (33)
6 - fluke (33)
7 - pluke (33)
8 - blude (32)
9 - elude (30)
Best guess is plume
Result is ['0', '2', '2', '0', '2']
Removed 5 words
Possible words remaining: 4
1 - blude (16)
2 - kluge (16)
3 - fluke (16)
4 - elude (15)
Best guess is fluke
Result is ['0', '2', '2', '0', '2']
Removed 2 words
Possible words remaining: 2
1 - blude (10)
2 - elude (9)
Best guess is blude
Result is ['0', '2', '2', '2', '2']
Removed 1 words
The word is elude
Guessed in 5
